In [12]:
import csv
import cv2
import numpy as np # numpy array is the format required by Keras

lines = []

with open('data/driving_log.csv') as csvDataFile:
    csvReader =csv.reader(csvDataFile)
    for line in csvReader:
        #print ("lines:", line)
        lines.append(line)
        
images = [] # this represents the features
measurements = [] # this represents the labels to predict

print ("there are %d lines in the file"%(len(lines)))

for line in lines[1:]: #skipping the headers
    
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = filename #'/data/IMG'+ filename
    #print("current path", current_path)
    image = cv2.imread(current_path)
    images.append(image)

  
    # extracting the steering wheel as labels
    #print ("steering angle", line[3])
    measurement = line[3] #float(line[3])
    
    #print ("measurement", measurement)
    measurements.append(measurement)

# checking array validity
print ("measurement in array", measurements[66])

    
# Data to feed the algorithm:

X_train = np.array(images)
print("X_train prepared, preparing y_train")
y_train = np.array(measurements)
print("y_train prepared")

from keras.models import Sequential
from keras.layers import Flatten, Dense

#Testing the data on a simplistic neural network
model = Sequential()
print("Sequential")
model.add(Flatten(input_shape=(160,320,3)))
print("Flatten")
model.add(Dense(1, activation='sigmoid'))
print("Dense")

# it is a regression network not a classification network, 'measn square errors' is used
model.compile(loss='mse', optimizer='adam')
# epoch is set to 7 because after this number the prediction accuracy decreases
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=7)
#model.fit(X_train, y_train,nb_epoch=20,batch_size=16)
#score = model.evaluate(X_train, y_train, batch_size=16)
model.save('model.h5')


there are 2591 lines in the file
measurement in array -0.0027
X_train prepared, preparing y_train
y_train prepared
Sequential
Flatten
Dense
Train on 2072 samples, validate on 518 samples
Epoch 1/7
2072/2072 [==============================] - 3s 1ms/step - loss: 0.0541 - val_loss: 0.0205